# Voxel Grid Creation

## Purpose

This notebook is **Step 1** of the AM-QADF workflow: Create separate empty voxel grids for each data source. Each source (laser, CT, ISPM, hatching) gets its own grid structure that will be used for signal mapping in the next step.

**Workflow Position**: After querying data (Notebook 01), before signal mapping (Notebook 03).

## Learning Objectives

By the end of this notebook, you will:
- ✅ Create separate empty grids for each data source (laser, CT, ISPM, hatching)
- ✅ Configure grid properties (bounding box, resolution, grid type) per source
- ✅ Use the grid naming module for consistent naming
- ✅ Understand that all grids must share the same bounding box
- ✅ Save each grid separately to MongoDB with proper naming

## Estimated Duration

60-90 minutes

---

## Overview

This notebook creates **separate empty voxel grids** for each data source. This is Step 1 of the correct workflow:

### Workflow Context

**Previous Step (Notebook 01)**: Query point cloud data per source
- ↓
**Current Step (Notebook 02)**: Create separate empty grids per source ← **YOU ARE HERE**
- ↓
**Next Step (Notebook 03)**: Map point cloud data to respective grids
- ↓
**Then (Notebook 04)**: Align grids temporally and spatially
- ↓
**Then (Notebook 05)**: Correct and calibrate each grid independently
- ↓
**Finally (Notebook 06)**: Fuse all corrected grids into one unified grid

### Key Requirements

1. **Separate Grids Per Source**:
   - **Grid A (Laser)**: Empty structure for laser parameters
   - **Grid B (CT)**: Empty structure for CT scan data
   - **Grid C (ISPM)**: Empty structure for ISPM monitoring data
   - **Grid D (Hatching)**: Empty structure for hatching path data

2. **Same Bounding Box**: All grids must use the **same bounding box** (from STL model or union of all data sources)

3. **Grid Types**: Each source can use different grid types:
   - 🧊 **Uniform**: Fixed resolution (recommended for initial implementation)
   - 🎯 **Adaptive**: Variable resolution based on spatial/temporal regions
   - 📊 **Multi-Resolution**: Hierarchical grids with multiple resolution levels

4. **Grid Naming**: Uses `am_qadf.voxel_domain.grid_naming` module:
   - Format: `{source}_{grid_type}_{resolution}_{stage}_{timestamp}`
   - Stage: `empty` (for newly created grids)
   - Example: `laser_uniform_50_empty_20250105_120000`

5. **Save Each Grid**: Each grid is saved separately to MongoDB with its own name

### Grid Creation Process

1. **Select Model**: Choose the STL model (determines bounding box)
2. **Select Source**: Choose which source to create grid for (laser, CT, ISPM, hatching)
3. **Configure Grid**: Set grid type, resolution, and properties
4. **Create Grid**: Generate empty grid structure
5. **Save Grid**: Save to MongoDB with proper naming

Use the interactive widgets below to create grids for each source - no coding required!

In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText,
    HTML as WidgetHTML
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from typing import Optional, Tuple, Dict, Any

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import infrastructure
INFRASTRUCTURE_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    INFRASTRUCTURE_AVAILABLE = True
except (ImportError, TypeError, Exception) as e:
    INFRASTRUCTURE_AVAILABLE = False
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}: {e}")

# Import grid naming module
try:
    from am_qadf.voxel_domain import GridNaming, GridSource, GridType, GridStage
    GRID_NAMING_AVAILABLE = True
except ImportError as e:
    GRID_NAMING_AVAILABLE = False
    print(f"⚠️ GridNaming not available: {e}")
    # Create dummy classes for compatibility
    class GridNaming:
        @staticmethod
        def generate_empty_grid_name(source, grid_type, resolution):
            return f"{source}_{grid_type}_{resolution}_empty_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    class GridSource:
        LASER = type('obj', (object,), {'value': 'laser'})()
        CT = type('obj', (object,), {'value': 'ct'})()
        ISPM = type('obj', (object,), {'value': 'ispm'})()
        HATCHING = type('obj', (object,), {'value': 'hatching'})()
    class GridType:
        UNIFORM = type('obj', (object,), {'value': 'uniform'})()
        ADAPTIVE = type('obj', (object,), {'value': 'adaptive'})()
        MULTIRES = type('obj', (object,), {'value': 'multi'})()
    class GridStage:
        EMPTY = type('obj', (object,), {'value': 'empty'})()

# Try to import SpatialQuery (needed for queries, but initialize client later)
try:
    from am_qadf.query import SpatialQuery
except ImportError:
    # Create dummy SpatialQuery for compatibility
    class SpatialQuery:
        def __init__(self, component_id=None, **kwargs):
            self.component_id = component_id

# Try to import voxelization classes
VOXEL_AVAILABLE = False
try:
    from am_qadf.voxelization.voxel_grid import VoxelGrid
    VOXEL_AVAILABLE = True
except ImportError:
    print("⚠️ VoxelGrid not available - using demo mode")

ADAPTIVE_AVAILABLE = False
try:
    from am_qadf.voxelization.adaptive_resolution import AdaptiveResolutionGrid
    ADAPTIVE_AVAILABLE = True
except ImportError:
    print("⚠️ AdaptiveResolutionGrid not available")

MULTI_AVAILABLE = False
try:
    from am_qadf.voxelization.multi_resolution import MultiResolutionGrid
    MULTI_AVAILABLE = True
except ImportError:
    print("⚠️ MultiResolutionGrid not available")

# Try to import query client for model selection
STL_CLIENT_AVAILABLE = False
try:
    from am_qadf.query import STLModelClient
    STL_CLIENT_AVAILABLE = True
except ImportError as e:
    STL_CLIENT_AVAILABLE = False
    print(f"⚠️ STLModelClient not available: {e}")

# Try to import alignment storage for loading aligned data
ALIGNMENT_STORAGE_AVAILABLE = False
alignment_storage = None
try:
    from am_qadf.synchronization import AlignmentStorage
    ALIGNMENT_STORAGE_AVAILABLE = True
except ImportError as e:
    ALIGNMENT_STORAGE_AVAILABLE = False
    print(f"⚠️ AlignmentStorage not available: {e}")

# Initialize MongoDB connection (optional, for model selection)
mongo_client = None
stl_client = None

if INFRASTRUCTURE_AVAILABLE and STL_CLIENT_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        
        if mongo_client and mongo_client.is_connected():
            stl_client = STLModelClient(mongo_client=mongo_client)
            # Initialize alignment storage if available
            if ALIGNMENT_STORAGE_AVAILABLE:
                try:
                    alignment_storage = AlignmentStorage(mongo_client=mongo_client)
                    print("✅ MongoDB connection established")
                    print("✅ AlignmentStorage initialized")
                except Exception as e:
                    print("✅ MongoDB connection established")
                    print(f"⚠️ AlignmentStorage initialization failed: {e}")
            else:
                print("✅ MongoDB connection established")
    except Exception as e:
        print(f"⚠️ MongoDB connection failed: {type(e).__name__}: {e}")

# Initialize unified query client (after mongo_client is initialized)
UNIFIED_CLIENT_AVAILABLE = False
unified_client = None
if mongo_client and mongo_client.is_connected():
    try:
        from am_qadf.query import UnifiedQueryClient
        unified_client = UnifiedQueryClient(mongo_client=mongo_client)
        UNIFIED_CLIENT_AVAILABLE = True
        print("✅ UnifiedQueryClient initialized")
    except ImportError as e:
        UNIFIED_CLIENT_AVAILABLE = False
        print(f"⚠️ UnifiedQueryClient not available: {e}")
    except Exception as e:
        UNIFIED_CLIENT_AVAILABLE = False
        print(f"⚠️ UnifiedQueryClient initialization failed: {e}")

# Try to import voxel grid storage (after mongo_client is initialized)
STORAGE_AVAILABLE = False
voxel_storage = None

try:
    # Import directly from voxel_storage to avoid importing voxel_domain_client
    # which has dependencies on signal_mapping that may not be available
    import importlib.util
    voxel_storage_path = src_dir / 'am_qadf' / 'voxel_domain' / 'voxel_storage.py'
    
    if voxel_storage_path.exists():
        spec = importlib.util.spec_from_file_location("voxel_storage", voxel_storage_path)
        voxel_storage_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(voxel_storage_module)
        VoxelGridStorage = voxel_storage_module.VoxelGridStorage
        STORAGE_AVAILABLE = True
        
        # Initialize storage only if mongo_client is available
        if mongo_client and mongo_client.is_connected():
            try:
                voxel_storage = VoxelGridStorage(mongo_client)
                print("✅ VoxelGridStorage initialized")
            except Exception as e:
                print(f"⚠️ Error initializing VoxelGridStorage: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("⚠️ VoxelGridStorage available but MongoDB not connected")
    else:
        raise ImportError(f"voxel_storage.py not found at {voxel_storage_path}")
        
except Exception as e:
    STORAGE_AVAILABLE = False
    print(f"⚠️ VoxelGridStorage not available: {e}")
    # Try fallback to regular import
    try:
        from am_qadf.voxel_domain.voxel_storage import VoxelGridStorage
        STORAGE_AVAILABLE = True
        if mongo_client and mongo_client.is_connected():
            voxel_storage = VoxelGridStorage(mongo_client)
            print("✅ VoxelGridStorage initialized (fallback import)")
    except Exception as e2:
        print(f"⚠️ Fallback import also failed: {e2}")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
⚠️ GridNaming not available: C++ OpenVDB bindings are required. Please build am_qadf_native with pybind11 bindings. Original error: cannot import name 'numpy_to_openvdb' from 'am_qadf_native' (unknown location)
⚠️ VoxelGrid not available - using demo mode
⚠️ AdaptiveResolutionGrid not available
⚠️ MultiResolutionGrid not available
⚠️ AlignmentStorage not available: cannot import name 'AlignmentStorage' from 'am_qadf.synchronization' (/mnt/c/Users/kanha/Independent_Research/AM-QADF/src/am_qadf/synchronization/__init__.py)
✅ MongoDB connection established
⚠️ UnifiedQueryClient not available: C++ bindings not available. Please build am_qadf_native with pybind11 bindings.
⚠️ VoxelGridStorage not available: No module named 'am_qadf_native.io'
⚠️ Fallback import also failed: C++ OpenVDB bindings are required. Please build am_qadf_native with pybind11 bindings. Original error: cannot import name 'numpy_to_openvdb' from 'am_qadf_native' (unkn

## Interactive Voxel Grid Creator

Use the widgets below to create and configure voxel grids. Select grid type, set bounding box and resolution, configure coordinate systems, and visualize the results!


In [2]:
# Create Interactive Voxel Grid Interface

# Global state
current_grid = None
current_grid_type = None
saved_grids = {}
current_grid_id = None  # Track the saved grid ID
current_grid_model_name = None  # Track the model name for current grid
current_grid_model_id = None  # Track the model ID for current grid
current_grid_name = None  # Track the generated grid name
operation_start_time = None  # Track operation timing

# ============================================
# Top Panel: Grid Type, Data Sources, and Actions
# ============================================

# Grid Type Selection
grid_type_label = widgets.HTML("<b>Grid Type:</b>")
grid_type = RadioButtons(
    options=[('Uniform', 'uniform'), ('Adaptive', 'adaptive'), ('Multi-Resolution', 'multi')],
    value='uniform',
    description='Type:',
    style={'description_width': 'initial'}
)

# Data Source Selection (Checkboxes - all selected by default, arranged in 2x2 grid)
data_source_label = widgets.HTML("<b>Data Source:</b>")
source_laser = Checkbox(value=True, description='Laser', style={'description_width': 'initial'})
source_ct = Checkbox(value=True, description='CT', style={'description_width': 'initial'})
source_ispm = Checkbox(value=True, description='ISPM', style={'description_width': 'initial'})
source_hatching = Checkbox(value=True, description='Hatching', style={'description_width': 'initial'})

# Store checkboxes in a list for easy access
source_checkboxes = [source_laser, source_ct, source_ispm, source_hatching]
source_mapping = {
    source_laser: 'laser',
    source_ct: 'ct',
    source_ispm: 'ispm',
    source_hatching: 'hatching'
}

# Helper function to get selected sources
def get_selected_sources():
    """Get list of selected data sources."""
    selected = []
    for checkbox, source in source_mapping.items():
        if checkbox.value:
            selected.append(source)
    return selected

# Source selection container - 2x2 grid layout
source_selection = VBox([
    HBox([source_laser, source_ct], layout=Layout(padding='2px')),
    HBox([source_ispm, source_hatching], layout=Layout(padding='2px'))
], layout=Layout(padding='5px'))

# Source dropdown for backward compatibility (used in create_grid for single grid creation)
source_dropdown = Dropdown(
    options=[
        ('Laser', 'laser'),
        ('CT Scan', 'ct'),
        ('ISPM', 'ispm'),
        ('Hatching', 'hatching')
    ],
    value='laser',
    description='Source:',
    style={'description_width': 'initial'},
    layout=Layout(display='none')  # Hidden, only used programmatically
)

# Sync source_dropdown with first selected checkbox
def update_source_dropdown_from_checkboxes(change=None):
    """Update source_dropdown based on checkbox selection."""
    selected = get_selected_sources()
    if selected:
        source_dropdown.value = selected[0]
    else:
        source_dropdown.value = 'laser'  # Default if nothing selected

# Update dropdown when checkboxes change
for checkbox in source_checkboxes:
    checkbox.observe(update_source_dropdown_from_checkboxes, names='value')

# Initialize dropdown
update_source_dropdown_from_checkboxes()

# Action Buttons (arranged in 2x2 grid)
create_button = Button(
    description='Create Grid',
    button_style='success',
    icon='plus',
    layout=Layout(width='120px'),
    tooltip='Create empty grids for all selected sources'  # Updated
)

load_button = Button(
    description='Load Grid',
    button_style='info',
    icon='folder-open',
    layout=Layout(width='120px'),
    tooltip='Load an existing empty grid'
)

save_button = Button(
    description='Save Grid',
    button_style='warning',
    icon='save',
    layout=Layout(width='120px'),
    tooltip='Save current grid to MongoDB'
)

# Action buttons container - 2x2 grid layout
action_buttons = VBox([
    HBox([create_button, load_button], layout=Layout(padding='2px')),
    HBox([save_button], layout=Layout(padding='2px'))
], layout=Layout(padding='5px'))

# ============================================
# Model and Grid Selection
# ============================================

# Model selector (for From Model mode)
# Load models from MongoDB
models = []
model_options = [("━━━ Select Model ━━━", None), ("━━━ All Models ━━━", "ALL")]

if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
        if len(model_options) == 2:  # Only "Select" and "All" options
            model_options.append(("No models available", None))
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")
        model_options.append(("Error loading models", None))
else:
    # Demo mode
    model_options.extend([
        ("Demo Model 1", "demo-001"),
        ("Demo Model 2", "demo-002")
    ])

# Set default model selection
default_model_value = None
if len(model_options) > 2:  # More than just "Select" and "All"
    # Default to "All Models" if available, otherwise first model
    if any(opt[1] == "ALL" for opt in model_options):
        default_model_value = "ALL"
    else:
        # Use first actual model (skip "Select Model" option)
        default_model_value = model_options[2][1] if len(model_options) > 2 else None
elif len(model_options) == 2 and any(opt[1] == "ALL" for opt in model_options):
    default_model_value = "ALL"

model_dropdown = Dropdown(
    options=model_options,
    value=default_model_value,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(display='flex', width='auto')
)

# Grid selector (for loading grids)
# Grid selector (for loading grids)
grid_dropdown = Dropdown(
    options=[("━━━ Select a Model First ━━━", None)],
    value=None,
    description='Select Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='350px')
)

# Refresh button to reload grids for selected model
refresh_grids_button = Button(
    description='Refresh Grids',
    button_style='info',
    icon='refresh',
    layout=Layout(width='120px'),
    tooltip='Refresh grid list for selected model'
)

# ============================================
# Model Info Display Widget
# ============================================

# Model info display (shows selected model details)
model_info_display = widgets.HTML(
    value="<i>No model selected</i>",
    layout=Layout(width='400px', padding='5px')
)

# Function to update model info display
def update_model_info_display():
    """Update the model info display when model selection changes."""
    if model_dropdown.value and model_dropdown.value != "ALL":
        # Extract model_id from dropdown (handle tuple or string)
        selected_model = model_dropdown.value
        if isinstance(selected_model, tuple):
            model_id = selected_model[1]
        else:
            model_id = selected_model
        
        # Find model name from models list
        model_name = "Unknown"
        for m in models:
            if m.get('model_id') == model_id:
                model_name = m.get('model_name') or m.get('filename') or m.get('original_stem', 'Unknown')
                break
        
        model_info_display.value = f"""
        <div style='background-color: #f0f0f0; padding: 8px; border-radius: 4px; border: 1px solid #ccc;'>
            <b>Selected Model:</b><br>
            <span style='color: #0066cc;'><b>{model_name}</b></span><br>
            <span style='color: #666; font-size: 0.9em;'>ID: {model_id}</span>
        </div>
        """
    elif model_dropdown.value == "ALL":
        model_info_display.value = """
        <div style='background-color: #fff3cd; padding: 8px; border-radius: 4px; border: 1px solid #ffc107;'>
            <b>All Models Selected</b><br>
            <span style='color: #666; font-size: 0.9em;'>Will create grids for all models</span>
        </div>
        """
    else:
        model_info_display.value = "<i style='color: #999;'>No model selected</i>"

# ============================================
# Top Panel Layout
# ============================================

top_panel = VBox([
    # First row: Model Selection, Grid Configuration, and Actions
    HBox([
        VBox([
            widgets.HTML("<b>📦 Model Selection:</b>"),
            model_dropdown,
            model_info_display
        ], layout=Layout(padding='5px', border='1px solid #4CAF50', flex='1')),
        VBox([
            widgets.HTML("<b>⚙️ Grid Configuration:</b>"),
            HBox([
                VBox([grid_type_label, grid_type], layout=Layout(padding='3px')),
                VBox([data_source_label, source_selection], layout=Layout(padding='3px'))
            ], layout=Layout(justify_content='flex-start'))
        ], layout=Layout(padding='5px', flex='1')),
        VBox([
            widgets.HTML("<b>🚀 Actions:</b>"),
            action_buttons
        ], layout=Layout(padding='5px'))
    ], layout=Layout(justify_content='space-between', padding='10px', border='1px solid #ccc', flex_wrap='wrap')),
    
    # Second row: Grid Selection (for loading existing grids)
    HBox([
        widgets.HTML("<b>📂 Select Grid to Load:</b>"),
        grid_dropdown,
        refresh_grids_button
    ], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ddd'))
])

# Connect refresh button
refresh_grids_button.on_click(lambda b: update_grid_dropdown())

# Bounding Box Section
bbox_label = widgets.HTML("<b>Bounding Box:</b>")
bbox_mode = RadioButtons(
    options=[
        ('From STL Model', 'model'),
        ('Union of Data Sources', 'union'),
        ('Custom', 'custom')
    ],
    value='model',  # Default to STL bounding box
    description='Mode:',
    style={'description_width': 'initial'}
)

# Custom bounding box sliders
bbox_x_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='X Min (mm):', style={'description_width': 'initial'})
bbox_x_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='X Max (mm):', style={'description_width': 'initial'})
bbox_y_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Min (mm):', style={'description_width': 'initial'})
bbox_y_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Max (mm):', style={'description_width': 'initial'})
bbox_z_min = FloatSlider(value=0.0, min=-1000.0, max=1000.0, step=1.0, description='Z Min (mm):', style={'description_width': 'initial'})
bbox_z_max = FloatSlider(value=100.0, min=-1000.0, max=1000.0, step=1.0, description='Z Max (mm):', style={'description_width': 'initial'})

bbox_sliders = VBox([
    bbox_x_min, bbox_x_max,
    bbox_y_min, bbox_y_max,
    bbox_z_min, bbox_z_max
])

def update_bbox_controls(change):
    """Show/hide bounding box controls based on mode."""
    if change['new'] == 'model':
        bbox_sliders.layout.display = 'none'
        # Update bbox sliders when model is selected (from top panel)
        if model_dropdown.value and model_dropdown.value != "ALL":
            update_bbox_from_model()
    elif change['new'] == 'union':
        bbox_sliders.layout.display = 'none'
    elif change['new'] == 'custom':
        bbox_sliders.layout.display = 'flex'
    else:  # interactive
        bbox_sliders.layout.display = 'none'
        
def update_bbox_from_model():
    """Update bounding box sliders from selected model."""
    # Extract model_id from dropdown (handle tuple or string)
    selected_model = model_dropdown.value
    if isinstance(selected_model, tuple):
        model_id = selected_model[1]
    else:
        model_id = selected_model
    
    if model_id and model_id != "ALL" and stl_client:
        try:
            model_data = stl_client.get_model(model_id)
            if model_data and 'metadata' in model_data:
                bbox = model_data['metadata'].get('bounding_box', {})
                if bbox and 'min' in bbox and 'max' in bbox:
                    bbox_min = bbox['min']
                    bbox_max = bbox['max']
                    # Update sliders (but don't trigger events)
                    bbox_x_min.value = bbox_min[0]
                    bbox_x_max.value = bbox_max[0]
                    bbox_y_min.value = bbox_min[1]
                    bbox_y_max.value = bbox_max[1]
                    bbox_z_min.value = bbox_min[2]
                    bbox_z_max.value = bbox_max[2]
        except Exception as e:
            print(f"⚠️ Error updating bbox from model: {e}")

# Also update bbox when model selection changes
def on_model_change(change):
    """When model changes, refresh grid list."""
    if change['new'] and change['new'] != "ALL":
        update_grid_dropdown()
    """Handle model selection change."""
    if bbox_mode.value == 'model' and change['new'] and change['new'] != "ALL":
        update_bbox_from_model()
    elif bbox_mode.value == 'model' and change['new'] == "ALL":
        # Update bbox for "All Models" case
        if stl_client and models:
            all_bboxes = []
            for m in models:
                model_id = m.get('model_id')
                if model_id:
                    try:
                        model_data = stl_client.get_model(model_id)
                        if model_data and 'metadata' in model_data:
                            bbox = model_data['metadata'].get('bounding_box', {})
                            if bbox and 'min' in bbox and 'max' in bbox:
                                all_bboxes.append((bbox['min'], bbox['max']))
                    except Exception as e:
                        print(f"⚠️ Error fetching model {model_id}: {e}")
            
            if all_bboxes:
                # Find union of all bounding boxes
                min_coords = [min(bbox[0][i] for bbox in all_bboxes) for i in range(3)]
                max_coords = [max(bbox[1][i] for bbox in all_bboxes) for i in range(3)]
                # Update sliders
                bbox_x_min.value = min_coords[0]
                bbox_x_max.value = max_coords[0]
                bbox_y_min.value = min_coords[1]
                bbox_y_max.value = max_coords[1]
                bbox_z_min.value = min_coords[2]
                bbox_z_max.value = max_coords[2]

model_dropdown.observe(on_model_change, names='value')

bbox_mode.observe(update_bbox_controls, names='value')

# Initialize bounding box from default selection
        # Initialize bounding box from default selection
if bbox_mode.value == 'model' and default_model_value:
    if default_model_value == "ALL":
        on_model_change({'new': "ALL"})
    else:
        update_bbox_from_model()
        
bbox_section = VBox([
    bbox_label,
    bbox_mode,
    bbox_sliders
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Resolution Section
resolution_label = widgets.HTML("<b>Resolution:</b>")
resolution_mode = RadioButtons(
    options=[('Uniform', 'uniform'), ('Per-Axis', 'per_axis'), ('Adaptive', 'adaptive')],
    value='uniform',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Uniform resolution
uniform_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='Resolution (mm):', style={'description_width': 'initial'})

# Per-axis resolution
x_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='X Res (mm):', style={'description_width': 'initial'})
y_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='Y Res (mm):', style={'description_width': 'initial'})
z_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='Z Res (mm):', style={'description_width': 'initial'})

per_axis_sliders = VBox([
    x_resolution, y_resolution, z_resolution
], layout=Layout(display='none'))

# Adaptive resolution config (collapsible)
adaptive_expand = Checkbox(value=False, description='Show Adaptive Config', style={'description_width': 'initial'})
adaptive_strategy = Dropdown(
    options=[('Data Density', 'density'), ('Gradient', 'gradient'), ('Custom Map', 'custom')],
    value='density',
    description='Strategy:',
    style={'description_width': 'initial'}
)

adaptive_config = VBox([
    adaptive_expand,
    adaptive_strategy
], layout=Layout(display='none'))

def update_resolution_controls(change):
    """Show/hide resolution controls based on mode."""
    if change['new'] == 'uniform':
        uniform_resolution.layout.display = 'flex'
        per_axis_sliders.layout.display = 'none'
        adaptive_config.layout.display = 'none'
    elif change['new'] == 'per_axis':
        uniform_resolution.layout.display = 'none'
        per_axis_sliders.layout.display = 'flex'
        adaptive_config.layout.display = 'none'
    else:  # adaptive
        uniform_resolution.layout.display = 'none'
        per_axis_sliders.layout.display = 'none'
        adaptive_config.layout.display = 'flex' if adaptive_expand.value else 'none'

resolution_mode.observe(update_resolution_controls, names='value')

def update_adaptive_display(change):
    """Show/hide adaptive config when expanded."""
    if resolution_mode.value == 'adaptive':
        adaptive_config.layout.display = 'flex' if change['new'] else 'none'

adaptive_expand.observe(update_adaptive_display, names='value')

resolution_section = VBox([
    resolution_label,
    resolution_mode,
    uniform_resolution,
    per_axis_sliders,
    adaptive_config
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Coordinate System Section
coord_label = widgets.HTML("<b>Coordinate System:</b>")
coord_system = Dropdown(
    options=[('Machine', 'machine'), ('Part', 'part'), ('Build Platform', 'build'), ('Custom', 'custom')],
    value='build',
    description='System:',
    style={'description_width': 'initial'}
)

origin_x = FloatText(value=0.0, description='Origin X:', style={'description_width': 'initial'})
origin_y = FloatText(value=0.0, description='Origin Y:', style={'description_width': 'initial'})
origin_z = FloatText(value=0.0, description='Origin Z:', style={'description_width': 'initial'})

rotation_x = FloatSlider(value=0.0, min=-180.0, max=180.0, step=1.0, description='Rot X (deg):', style={'description_width': 'initial'})
rotation_y = FloatSlider(value=0.0, min=-180.0, max=180.0, step=1.0, description='Rot Y (deg):', style={'description_width': 'initial'})
rotation_z = FloatSlider(value=0.0, min=-180.0, max=180.0, step=1.0, description='Rot Z (deg):', style={'description_width': 'initial'})

transform_preview = Button(description='Preview Transform', button_style='', layout=Layout(width='150px'))

coord_section = VBox([
    coord_label,
    coord_system,
    origin_x, origin_y, origin_z,
    rotation_x, rotation_y, rotation_z,
    transform_preview
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Grid Properties Section
props_label = widgets.HTML("<b>Grid Properties:</b>")
aggregation_method = Dropdown(
    options=[('mean', 'mean'), ('max', 'max'), ('min', 'min'), ('sum', 'sum'), ('median', 'median')],
    value='mean',
    description='Aggregation:',
    style={'description_width': 'initial'}
)

sparse_storage = Checkbox(value=True, description='Sparse Storage', style={'description_width': 'initial'})
compression = Checkbox(value=False, description='Compression', style={'description_width': 'initial'})

props_section = VBox([
    props_label,
    aggregation_method,
    sparse_storage,
    compression
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Left panel assembly
left_panel = VBox([
    bbox_section,
    resolution_section,
    coord_section,
    props_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Grid Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('3D View', '3d'), ('2D Slices', '2d'), ('Properties', 'props')],
    value='props',
    description='View:',
    style={'description_width': 'initial'}
)

# 3D viewer output
viewer_3d = Output(layout=Layout(height='400px', overflow='auto'))

# 2D slice controls
slice_axis = RadioButtons(
    options=[('XY', 'xy'), ('XZ', 'xz'), ('YZ', 'yz')],
    value='xy',
    description='Axis:',
    style={'description_width': 'initial'}
)

slice_position = IntSlider(value=0, min=0, max=100, step=1, description='Position:', style={'description_width': 'initial'})
slice_viewer = Output(layout=Layout(height='400px'))

slice_controls = VBox([
    slice_axis,
    slice_position,
    slice_viewer
], layout=Layout(display='none'))

# Properties display
props_display = Output(layout=Layout(height='400px', overflow='auto'))

def update_viz_display(change):
    """Show/hide visualization controls based on mode."""
    if change['new'] == '3d':
        viewer_3d.layout.display = 'flex'
        slice_controls.layout.display = 'none'
        props_display.layout.display = 'none'
    elif change['new'] == '2d':
        viewer_3d.layout.display = 'none'
        slice_controls.layout.display = 'flex'
        props_display.layout.display = 'none'
    else:  # props
        viewer_3d.layout.display = 'none'
        slice_controls.layout.display = 'none'
        props_display.layout.display = 'flex'

viz_mode.observe(update_viz_display, names='value')

center_panel = VBox([
    widgets.HTML("<h3>Grid Visualization</h3>"),
    viz_mode,
    viewer_3d,
    slice_controls,
    props_display
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Grid Statistics & Actions
# ============================================

# Grid Statistics
stats_label = widgets.HTML("<b>Grid Statistics:</b>")
stats_display = widgets.HTML("No grid created yet")
stats_section = VBox([
    stats_label,
    stats_display
], layout=Layout(padding='5px'))

# Grid Metadata
metadata_label = widgets.HTML("<b>Grid Metadata:</b>")
metadata_display = widgets.HTML("No metadata available")
metadata_section = VBox([
    metadata_label,
    metadata_display
], layout=Layout(padding='5px'))

# Quick Actions
actions_label = widgets.HTML("<b>Quick Actions:</b>")
export_button = Button(description='Export Grid', button_style='', icon='download', layout=Layout(width='150px'))
validate_button = Button(description='Validate Grid', button_style='', icon='check', layout=Layout(width='150px'))
compare_button = Button(description='Compare Grids', button_style='', icon='copy', layout=Layout(width='150px'))

actions_section = VBox([
    actions_label,
    export_button,
    validate_button,
    compare_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    stats_section,
    metadata_section,
    actions_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

# Status display widget
current_operation = WidgetHTML(value='<b>Status:</b> Ready to create grid')

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Grid creation logs output
grid_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with grid_logs:
    display(HTML("<p><i>Grid creation logs will appear here...</i></p>"))

# Bottom status bar (shows Status | Progress | Time)
bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Error display (kept for backward compatibility)
error_display = widgets.HTML("")

# Enhanced bottom panel
bottom_panel = VBox([
    current_operation,
    progress_bar,
    WidgetHTML("<b>Grid Creation Logs:</b>"),
    grid_logs,
    WidgetHTML("<hr>"),
    bottom_status,
    bottom_progress,
    error_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# Keep old status_display for backward compatibility (will be updated by logging functions)
status_display = current_operation

# Global time tracking
operation_start_time = None

# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the grid creation logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with grid_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display and progress."""
    global operation_start_time
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(elapsed))}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> 0:00'

# ============================================
# Grid Creation Functions
# ============================================

def create_grid(button):
    """Create empty grids for all selected sources."""
    global current_grid, current_grid_type, operation_start_time, current_grid_name
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with grid_logs:
        clear_output(wait=True)
    
    log_message("Starting grid creation...", 'info')
    update_status("Initializing grid creation...", 0)
    error_display.value = ""
    
    # Safety check: ensure required widgets exist
    try:
        if 'bbox_mode' not in globals() or bbox_mode is None:
            log_message("Error: bbox_mode widget not initialized", 'error')
            error_display.value = "<span style='color: red;'>❌ Error: bbox_mode widget not initialized. Please restart the notebook.</span>"
            update_status("Error: Missing widget", 0)
            return
    except NameError:
        log_message("Error: bbox_mode widget not defined", 'error')
        error_display.value = "<span style='color: red;'>❌ Error: bbox_mode widget not defined. Please restart the notebook.</span>"
        update_status("Error: Missing widget", 0)
        return
    
    # Check if grid naming is available
    if not GRID_NAMING_AVAILABLE:
        log_message("Grid naming module not available", 'warning')
        error_display.value = "<span style='color: orange;'>⚠️ Grid naming module not available. Grid will not have proper naming.</span>"
    
    # Get selected sources from checkboxes
    selected_sources = get_selected_sources()
    
    if not selected_sources:
        log_message("Please select at least one data source", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select at least one data source</span>"
        update_status("No source selected", 0)
        return
    
    # Get model_id for grid naming
    model_id = None
    if model_dropdown.value and model_dropdown.value != "ALL":
        # Handle tuple (display_name, model_id) or direct model_id string
        if isinstance(model_dropdown.value, tuple):
            model_id = model_dropdown.value[1]  # Extract model_id from tuple
        else:
            model_id = model_dropdown.value
    
    if not model_id:
        log_message("Please select a model first", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select a model first</span>"
        update_status("No model selected", 0)
        return
    
    log_message(f"Creating grids for {len(selected_sources)} selected source(s): {', '.join(selected_sources)}", 'info')
    log_message(f"Model ID: {model_id}", 'info')
    
    try:
        # Get bounding box
        log_message("Fetching bounding box...", 'info')
        update_status("Fetching bounding box...", 10)
        
        if bbox_mode.value == 'union' or bbox_mode.value == 'data':
            # Union of all data sources (min/max of all point cloud data)
            log_message("Calculating union bounding box from all data sources...", 'info')
            update_status("Querying data from all sources...", 15)
            
            all_points = []
            sources_queried = []
            
            # Query data from all sources
            if unified_client and mongo_client:
                # Query Laser data
                try:
                    log_message("Querying laser parameters...", 'info')
                    if unified_client.laser_client and model_id:
                        spatial_query = SpatialQuery(component_id=model_id)
                        laser_result = unified_client.laser_client.query(spatial=spatial_query)
                        if laser_result and hasattr(laser_result, 'points') and laser_result.points:
                            all_points.append(np.array(laser_result.points))
                            sources_queried.append('laser')
                            log_message(f"✅ Laser: {len(laser_result.points)} points", 'success')
                except Exception as e:
                    log_message(f"Error querying laser: {e}", 'warning')
                
                # Query CT data
                try:
                    log_message("Querying CT scan data...", 'info')
                    if unified_client.ct_client and model_id:
                        ct_data = unified_client.ct_client.get_scan(model_id)
                        if ct_data and 'points' in ct_data:
                            points = np.array(ct_data['points'])
                            if len(points) > 0:
                                all_points.append(points)
                                sources_queried.append('ct')
                                log_message(f"✅ CT: {len(points)} points", 'success')
                except Exception as e:
                    log_message(f"Error querying CT: {e}", 'warning')
                
                # Query ISPM data
                try:
                    log_message("Querying ISPM monitoring data...", 'info')
                    if unified_client.ispm_client and model_id:
                        spatial_query = SpatialQuery(component_id=model_id)
                        ispm_result = unified_client.ispm_client.query(spatial=spatial_query)
                        if ispm_result and hasattr(ispm_result, 'points') and ispm_result.points:
                            all_points.append(np.array(ispm_result.points))
                            sources_queried.append('ispm')
                            log_message(f"✅ ISPM: {len(ispm_result.points)} points", 'success')
                except Exception as e:
                    log_message(f"Error querying ISPM: {e}", 'warning')
                
                # Query Hatching data
                try:
                    log_message("Querying hatching layers...", 'info')
                    if unified_client.hatching_client and model_id:
                        layers = unified_client.hatching_client.get_layers(model_id)
                        if layers:
                            hatching_points = []
                            for layer in layers:
                                if 'hatches' in layer:
                                    for hatch in layer['hatches']:
                                        if 'points' in hatch and len(hatch['points']) > 0:
                                            hatching_points.extend(hatch['points'])
                            if hatching_points:
                                all_points.append(np.array(hatching_points))
                                sources_queried.append('hatching')
                                log_message(f"✅ Hatching: {len(hatching_points)} points", 'success')
                except Exception as e:
                    log_message(f"Error querying hatching: {e}", 'warning')
            
            # Calculate union bounding box
            if all_points:
                combined_points = np.vstack(all_points)
                bbox_min = tuple(combined_points.min(axis=0))
                bbox_max = tuple(combined_points.max(axis=0))
                log_message(f"✅ Union bounding box: {bbox_min} to {bbox_max}", 'success')
                log_message(f"   Sources included: {', '.join(sources_queried)}", 'info')
                log_message(f"   Total points: {len(combined_points):,}", 'info')
            else:
                log_message("⚠️ No data points found, falling back to STL bounding box", 'warning')
                # Fallback to STL bbox
                if stl_client and model_id:
                    try:
                        model_data = stl_client.get_model(model_id)
                        if model_data and 'metadata' in model_data:
                            bbox = model_data['metadata'].get('bounding_box', {})
                            if bbox and 'min' in bbox and 'max' in bbox:
                                bbox_min = tuple(bbox['min'])
                                bbox_max = tuple(bbox['max'])
                    except:
                        pass
                
                # Final fallback to custom
                if 'bbox_min' not in locals() or bbox_min is None:
                    bbox_min = (bbox_x_min.value, bbox_y_min.value, bbox_z_min.value)
                    bbox_max = (bbox_x_max.value, bbox_y_max.value, bbox_z_max.value)
        elif bbox_mode.value == 'model' and model_dropdown.value:
            # Extract model_id from dropdown (handle tuple or string)
            selected_model = model_dropdown.value
            if isinstance(selected_model, tuple):
                selected_model_id = selected_model[1]
            else:
                selected_model_id = selected_model
            
            if selected_model_id == "ALL":
                # For "All Models", use union of all bounding boxes
                if stl_client and models:
                    all_bboxes = []
                    for m in models:
                        m_id = m.get('model_id')
                        if m_id:
                            try:
                                model_data = stl_client.get_model(m_id)
                                if model_data and 'metadata' in model_data:
                                    bbox = model_data['metadata'].get('bounding_box', {})
                                    if bbox and 'min' in bbox and 'max' in bbox:
                                        all_bboxes.append((bbox['min'], bbox['max']))
                            except Exception as e:
                                print(f"⚠️ Error fetching model {m_id}: {e}")
                    
                    if all_bboxes:
                        # Find union of all bounding boxes
                        min_coords = [min(bbox[0][i] for bbox in all_bboxes) for i in range(3)]
                        max_coords = [max(bbox[1][i] for bbox in all_bboxes) for i in range(3)]
                        bbox_min = tuple(min_coords)
                        bbox_max = tuple(max_coords)
                    else:
                        # Fallback
                        bbox_min = (-50.0, -50.0, 0.0)
                        bbox_max = (50.0, 50.0, 100.0)
                else:
                    # Demo mode fallback
                    bbox_min = (-50.0, -50.0, 0.0)
                    bbox_max = (50.0, 50.0, 100.0)
            else:
                # Single model selected
                if stl_client and model_id:
                    try:
                        model_data = stl_client.get_model(model_id)
                        if model_data and 'metadata' in model_data:
                            bbox = model_data['metadata'].get('bounding_box', {})
                            if bbox and 'min' in bbox and 'max' in bbox:
                                bbox_min = tuple(bbox['min'])
                                bbox_max = tuple(bbox['max'])
                            else:
                                # Fallback if bbox not found
                                bbox_min = (-50.0, -50.0, 0.0)
                                bbox_max = (50.0, 50.0, 100.0)
                        else:
                            bbox_min = (-50.0, -50.0, 0.0)
                            bbox_max = (50.0, 50.0, 100.0)
                    except Exception as e:
                        error_display.value = f"<span style='color: orange;'>⚠️ Error fetching model: {e}. Using default bbox.</span>"
                        bbox_min = (-50.0, -50.0, 0.0)
                        bbox_max = (50.0, 50.0, 100.0)
                else:
                    # Demo mode
                    bbox_min = (-50.0, -50.0, 0.0)
                    bbox_max = (50.0, 50.0, 100.0)
        elif bbox_mode.value == 'custom':
            bbox_min = (bbox_x_min.value, bbox_y_min.value, bbox_z_min.value)
            bbox_max = (bbox_x_max.value, bbox_y_max.value, bbox_z_max.value)
        else:
            bbox_min = (-50.0, -50.0, 0.0)
            bbox_max = (50.0, 50.0, 100.0)
        
        status_display.value = "<b>Status:</b> Calculating bounding box..."
        progress_bar.value = 30
        time.sleep(0.1)
        
        # Get resolution
        status_display.value = "<b>Status:</b> Setting resolution..."
        progress_bar.value = 40
        time.sleep(0.1)
        
        if resolution_mode.value == 'uniform':
            resolution = uniform_resolution.value
        elif resolution_mode.value == 'per_axis':
            # Use average for uniform grids, or create per-axis grid if supported
            resolution = (x_resolution.value + y_resolution.value + z_resolution.value) / 3.0
        else:
            resolution = uniform_resolution.value  # Default for adaptive
        
        # Get grid type
        grid_type_val = grid_type.value
        
        # Map selected sources to GridSource enums
        source_enum_map = {
            'laser': GridSource.LASER,
            'ct': GridSource.CT,
            'ispm': GridSource.ISPM,
            'hatching': GridSource.HATCHING
        }
        
        # Create list of (source_name, source_enum) tuples for selected sources only
        sources = [(source, source_enum_map[source]) for source in selected_sources if source in source_enum_map]
        
        if not sources:
            log_message("No valid sources selected", 'error')
            error_display.value = "<span style='color: red;'>❌ No valid sources selected</span>"
            update_status("Invalid sources", 0)
            return
        
        created_grids = {}
        saved_grid_ids = {}
        
        log_message(f"Creating {len(sources)} empty grids...", 'info')
        update_status(f"Creating {len(sources)} grids...", 50)
        
        for idx, (source_name, source_enum) in enumerate(sources):
            try:
                log_message(f"Creating {source_name} grid ({idx+1}/{len(sources)})...", 'info')
                update_status(f"Creating {source_name} grid...", 50 + (idx * 10))
                
                # Generate grid name using GridNaming module
                grid_name = GridNaming.generate_empty_grid_name(
                    source=source_enum.value,
                    grid_type=grid_type_val,
                    resolution=resolution
                )
                
                log_message(f"Grid name: {grid_name}", 'info')
                
                # Create grid based on type
                if grid_type_val == 'uniform' and VOXEL_AVAILABLE:
                    grid = VoxelGrid(
                        bbox_min=bbox_min,
                        bbox_max=bbox_max,
                        resolution=resolution,
                        aggregation=aggregation_method.value
                    )
                elif grid_type_val == 'adaptive' and ADAPTIVE_AVAILABLE:
                    grid = AdaptiveResolutionGrid(
                        bbox_min=bbox_min,
                        bbox_max=bbox_max,
                        base_resolution=resolution
                    )
                elif grid_type_val == 'multi' and MULTI_AVAILABLE:
                    grid = MultiResolutionGrid(
                        bbox_min=bbox_min,
                        bbox_max=bbox_max,
                        base_resolution=resolution,
                        num_levels=3,
                        level_ratio=2.0
                    )
                else:
                    # Demo mode - create a simple representation
                    class DemoGrid:
                        def __init__(self, bbox_min, bbox_max, resolution):
                            self.bbox_min = np.array(bbox_min)
                            self.bbox_max = np.array(bbox_max)
                            self.resolution = resolution
                            self.size = self.bbox_max - self.bbox_min
                            self.dims = np.ceil(self.size / resolution).astype(int)
                            self.dims = np.maximum(self.dims, [1, 1, 1])
                            self.actual_size = self.dims * resolution
                            self.aggregation = aggregation_method.value
                            self.grid_type = grid_type_val
                            self.created_at = datetime.now()
                    
                    grid = DemoGrid(bbox_min, bbox_max, resolution)
                
                # Store grid metadata
                if not hasattr(grid, 'metadata'):
                    grid.metadata = {}
                
                grid.metadata['source'] = source_name
                grid.metadata['grid_name'] = grid_name
                grid.metadata['model_id'] = model_id
                
                created_grids[source_name] = {
                    'grid': grid,
                    'grid_name': grid_name,
                    'source': source_name
                }
                
                log_message(f"✅ {source_name} grid created: {grid_name}", 'success')
                
            except Exception as e:
                log_message(f"Error creating {source_name} grid: {e}", 'error')
                import traceback
                log_message(f"Traceback: {traceback.format_exc()}", 'error')
                continue
        
        # Store created grids
        saved_grids.update(created_grids)
        
        # Set current grid to last created
        if created_grids:
            last_source = list(created_grids.keys())[-1]
            current_grid = created_grids[last_source]['grid']
            current_grid_type = grid_type_val
            current_grid_name = created_grids[last_source]['grid_name']
            log_message(f"Current grid set to: {last_source}", 'info')
        
        # Show summary
        grid_info = f"""
        <p><b>✅ Created {len(created_grids)} Empty Grid(s):</b></p>
        <ul>
        """
        for source_name, grid_info_dict in created_grids.items():
            grid_info += f"<li><b>{source_name.upper()}</b>: {grid_info_dict['grid_name']}</li>"
        grid_info += f"""
        </ul>
        <p><b>Grid Type:</b> {grid_type_val}</p>
        <p><b>Resolution:</b> {resolution} mm</p>
        <p><b>Bounding Box:</b> {bbox_min} to {bbox_max}</p>
        <p><b>💡 Click 'Save Grid' to save these empty grids to MongoDB.</b></p>
        """
        status_display.value = grid_info
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Grids created successfully in {total_time:.2f}s", 'success')
        else:
            log_message("Grids created successfully", 'success')
        
        # Update displays
        update_grid_displays()
        
        # Update visualizations
        if viz_mode.value == '3d':
            update_3d_view()
        elif viz_mode.value == '2d':
            update_slice_view()
        else:
            update_properties_display()
        
        update_status("Grids created successfully", 100)
        
    except Exception as e:
        log_message(f"Error creating grids: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        update_status("Error creating grids", 0)
        
        
def update_grid_displays():
    """Update all grid-related displays."""
    global current_grid
    
    if current_grid is None:
        return
    
    # Update statistics
    if hasattr(current_grid, 'dims'):
        dims = current_grid.dims
        total_voxels = int(np.prod(dims))
        memory_mb = total_voxels * 8 / (1024 * 1024)  # Rough estimate
        
        stats_html = f"""
        <p><b>Dimensions:</b> {dims[0]} × {dims[1]} × {dims[2]}</p>
        <p><b>Total Voxels:</b> {total_voxels:,}</p>
        <p><b>Memory (est.):</b> {memory_mb:.2f} MB</p>
        <p><b>Resolution:</b> {current_grid.resolution if hasattr(current_grid, 'resolution') else 'N/A'} mm</p>
        """
    else:
        stats_html = "<p>Statistics not available</p>"
    
    stats_display.value = stats_html
    
    # Update metadata
    metadata_html = f"""
    <p><b>Type:</b> {current_grid_type}</p>
    <p><b>Created:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    <p><b>Coordinate System:</b> {coord_system.value}</p>
    <p><b>Aggregation:</b> {aggregation_method.value}</p>
    <p><b>Storage:</b> {'Sparse' if sparse_storage.value else 'Dense'}</p>
    """
    metadata_display.value = metadata_html
    
    # Update properties display
    with props_display:
        clear_output(wait=True)
        
        # Format bounding box values nicely
        def format_bbox(bbox):
            if bbox is None or not hasattr(current_grid, 'bbox_min'):
                return 'N/A'
            if isinstance(bbox, np.ndarray):
                # Format each value with 2 decimal places, using decimal notation
                # For very small numbers, show as 0.00 instead of scientific notation
                formatted = []
                for val in bbox:
                    if abs(val) < 1e-10:  # Essentially zero
                        formatted.append("0.00")
                    elif abs(val) < 0.01:  # Very small but not zero
                        formatted.append(f"{val:.4f}")
                    else:
                        formatted.append(f"{val:.2f}")
                return f"[{', '.join(formatted)}]"
            elif isinstance(bbox, (list, tuple)):
                formatted = []
                for val in bbox:
                    if abs(val) < 1e-10:
                        formatted.append("0.00")
                    elif abs(val) < 0.01:
                        formatted.append(f"{val:.4f}")
                    else:
                        formatted.append(f"{val:.2f}")
                return f"[{', '.join(formatted)}]"
            return str(bbox)
        
        # Format dimensions
        def format_dims(dims):
            if dims is None or not hasattr(current_grid, 'dims'):
                return 'N/A'
            if isinstance(dims, np.ndarray):
                return f"[{', '.join(map(str, dims.astype(int)))}]"
            elif isinstance(dims, (list, tuple)):
                return f"[{', '.join(map(str, map(int, dims)))}]"
            return str(dims)
        
        # Get model name from current selection or saved grid info
        display_model_name = current_grid_model_name
        if not display_model_name and model_dropdown.value and model_dropdown.value != "ALL":
            # Try to get from selected model
            for m in models:
                if m.get('model_id') == model_dropdown.value:
                    display_model_name = m.get('model_name') or m.get('filename') or m.get('original_stem', 'Unknown')
                    break
        
        # Get grid ID (if saved)
        display_grid_id = current_grid_id[:8] + "..." if current_grid_id else "Not saved"
        
        bbox_min_str = format_bbox(current_grid.bbox_min if hasattr(current_grid, 'bbox_min') else None)
        bbox_max_str = format_bbox(current_grid.bbox_max if hasattr(current_grid, 'bbox_max') else None)
        dims_str = format_dims(current_grid.dims if hasattr(current_grid, 'dims') else None)
        resolution_str = f"{current_grid.resolution:.2f}" if hasattr(current_grid, 'resolution') else 'N/A'
        total_voxels = int(np.prod(current_grid.dims)) if hasattr(current_grid, 'dims') else 'N/A'
        
        # Get resolution mode and values
        resolution_mode_str = resolution_mode.value if hasattr(resolution_mode, 'value') else 'N/A'
        if resolution_mode_str == 'uniform':
            resolution_detail = f"{uniform_resolution.value:.2f} mm (uniform)"
        elif resolution_mode_str == 'per_axis':
            resolution_detail = f"X: {x_resolution.value:.2f}, Y: {y_resolution.value:.2f}, Z: {z_resolution.value:.2f} mm"
        else:
            resolution_detail = f"{uniform_resolution.value:.2f} mm (adaptive)"
        
        # Get coordinate system details
        coord_origin = f"[{origin_x.value:.2f}, {origin_y.value:.2f}, {origin_z.value:.2f}]"
        coord_rotation = f"[{rotation_x.value:.1f}°, {rotation_y.value:.1f}°, {rotation_z.value:.1f}°]"
        
        html = f"""
        <h4>Grid Properties</h4>
        <table border='1' style='border-collapse: collapse; width: 100%;'>
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Grid Information</b></th></tr>
        <tr><td><b>Grid ID</b></td><td>{display_grid_id}</td></tr>
        <tr><td><b>Model Name</b></td><td>{display_model_name if display_model_name else 'N/A'}</td></tr>
        <tr><td><b>Model ID</b></td><td>{current_grid_model_id[:8] + '...' if current_grid_model_id else 'N/A'}</td></tr>
        <tr><td><b>Grid Type</b></td><td>{current_grid_type if current_grid_type else 'N/A'}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Spatial Properties</b></th></tr>
        <tr><td>Bounding Box Min</td><td>{bbox_min_str}</td></tr>
        <tr><td>Bounding Box Max</td><td>{bbox_max_str}</td></tr>
        <tr><td>Bounding Box Source</td><td>{bbox_mode.value if hasattr(bbox_mode, 'value') else 'N/A'}</td></tr>
        <tr><td>Dimensions</td><td>{dims_str}</td></tr>
        <tr><td>Total Voxels</td><td>{total_voxels:,}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Resolution Settings</b></th></tr>
        <tr><td>Resolution Mode</td><td>{resolution_mode_str}</td></tr>
        <tr><td>Resolution</td><td>{resolution_detail}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Coordinate System</b></th></tr>
        <tr><td>System Type</td><td>{coord_system.value if hasattr(coord_system, 'value') else 'N/A'}</td></tr>
        <tr><td>Origin</td><td>{coord_origin}</td></tr>
        <tr><td>Rotation</td><td>{coord_rotation}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Grid Properties</b></th></tr>
        <tr><td>Aggregation Method</td><td>{aggregation_method.value}</td></tr>
        <tr><td>Sparse Storage</td><td>{'Yes' if sparse_storage.value else 'No'}</td></tr>
        <tr><td>Compression</td><td>{'Yes' if compression.value else 'No'}</td></tr>
        </table>
        """
        display(HTML(html))

def update_properties_display():
    """Update the properties display tab with current grid information."""
    update_grid_displays()  # This already updates the properties display

def update_slice_view():
    """Update 2D slice visualization for empty grids using PyVista."""
    global current_grid
    
    if current_grid is None or not hasattr(current_grid, 'dims'):
        with slice_viewer:
            clear_output(wait=True)
            display(HTML("<p style='color:orange;'>⚠️ No grid created yet.</p>"))
        return
    
    with slice_viewer:
        clear_output(wait=True)
        display(HTML("<p>🔄 Generating 2D slice visualization...</p>"))
        
        try:
            dims = current_grid.dims
            axis_str = slice_axis.value  # 'xy', 'xz', or 'yz'
            pos = slice_position.value  # 0-100 (percentage)
            
            # Map notebook axis values to PyVista axis values
            axis_map = {
                'xy': 'z',  # XY slice = slice along Z axis
                'xz': 'y',  # XZ slice = slice along Y axis
                'yz': 'x'   # YZ slice = slice along X axis
            }
            axis = axis_map.get(axis_str, 'z')
            
            # Convert position from 0-100 to 0.0-1.0 (normalized)
            position = pos / 100.0
            
            # Use GridVisualizer for empty grid visualization
            from am_qadf.visualization.grid_visualizer import GridVisualizer
            
            visualizer = GridVisualizer()
            
            # Plot slice using PyVista (no signal needed for empty grids)
            plotter = visualizer.plot_grid_slice(
                voxel_grid=current_grid,
                axis=axis,
                position=position,
                signal_name=None,  # Empty grid - no signals
                colormap='RdYlBu',
                show_grid_lines=True,
                notebook=True,
                auto_show=True
            )
            
            if plotter is None:
                display(HTML("<p style='color:red;'>❌ Error: Could not create PyVista slice visualization</p>"))
                return
            
            # Get source from metadata
            source = current_grid.metadata.get('source', 'unknown').upper() if hasattr(current_grid, 'metadata') else 'unknown'
            
            signal_info = f"<p>✅ <b>Empty {source} grid structure visualized</b></p>"
            signal_info += f"<p>Slice: <b>{axis_str.upper()}</b> plane at <b>{pos}%</b> position</p>"
            signal_info += f"<p>Resolution: <b>{current_grid.resolution:.3f} mm</b></p>"
            signal_info += f"<p>Grid dimensions: {dims[0]}×{dims[1]}×{dims[2]} voxels</p>"
            signal_info += "<p>Checkerboard pattern shows voxel boundaries clearly.</p>"
            signal_info += "<p>💡 <i>Tip: Adjust position slider to see different slices</i></p>"
            signal_info += "<p>💡 <i>Next: Go to Notebook 03 to map signals to this grid</i></p>"
            
            display(HTML(signal_info))
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
            
# ============================================
# 3D Grid Visualization Function
# ============================================
def update_3d_view():
    """Update 3D grid visualization for empty grids using GridVisualizer (PyVista)."""
    global current_grid
    
    if current_grid is None or not hasattr(current_grid, 'dims'):
        with viewer_3d:
            clear_output(wait=True)
            display(HTML("<p style='color:orange;'>⚠️ No grid created yet. Create a grid first.</p>"))
        return
    
    with viewer_3d:
        clear_output(wait=True)
        display(HTML("<p>🔄 Generating 3D visualization...</p>"))
        
        try:
            # For Notebook 02, we only visualize empty grids
            from am_qadf.visualization.grid_visualizer import GridVisualizer
            
            visualizer = GridVisualizer()
            
            # Plot empty grid structure using PyVista
            plotter = visualizer.plot_grid_structure(
                voxel_grid=current_grid,
                show_edges=True,
                edge_color='black',
                face_color='lightblue',
                opacity=0.7,
                show_grid_outline=True,
                notebook=True,
                auto_show=True
            )
            
            if plotter is None:
                display(HTML("<p style='color:red;'>❌ Error: Could not create PyVista visualization</p>"))
                return
            
            # Get grid info
            dims = current_grid.dims if hasattr(current_grid, 'dims') else [0, 0, 0]
            resolution = current_grid.resolution if hasattr(current_grid, 'resolution') else 0.0
            total_voxels = int(np.prod(dims)) if hasattr(current_grid, 'dims') else 0
            
            # Get source from metadata
            source = current_grid.metadata.get('source', 'unknown').upper() if hasattr(current_grid, 'metadata') else 'unknown'
            
            signal_info = f"<p>✅ <b>Empty {source} grid structure visualized</b></p>"
            signal_info += f"<p>Grid dimensions: {dims[0]}×{dims[1]}×{dims[2]} = {total_voxels:,} voxels</p>"
            signal_info += f"<p><b>Resolution: {resolution:.3f} mm</b></p>"
            signal_info += "<p>This visualization shows the empty grid structure with visible voxel edges.</p>"
            signal_info += "<p>💡 <i>Tip: Rotate, zoom, and pan the 3D view to see voxel structure clearly</i></p>"
            signal_info += "<p>💡 <i>Next: Go to Notebook 03 to map signals to this grid</i></p>"
            
            display(HTML(signal_info))
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
            
# Update 3D view when grid is created or mode changes
def on_viz_mode_change(change):
    """Handle visualization mode change."""
    if change['new'] == '3d':
        update_3d_view()

# Connect the observer
viz_mode.observe(on_viz_mode_change, names='value')


# ============================================
# Save Grid Function
# ============================================
def save_grid(button):
    """Save all created grids to MongoDB."""
    global current_grid, operation_start_time, current_grid_name, current_grid_id, current_grid_model_id, current_grid_model_name, saved_grids
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with grid_logs:
        clear_output(wait=True)
    
    log_message("Starting grid save operation...", 'info')
    update_status("Initializing save...", 0)
    
    # Check if there are any grids to save
    if not saved_grids:
        log_message("No grids to save. Create grids first.", 'warning')
        error_display.value = "<span style='color: red;'>❌ No grids to save. Create grids first.</span>"
        update_status("No grids to save", 0)
        return
    
    # Filter out grids that have already been saved (have grid_id)
    grids_to_save = []
    for source_name, grid_info in saved_grids.items():
        # Check if this grid has already been saved (has grid_id)
        if isinstance(grid_info, dict) and 'grid_id' in grid_info:
            continue  # Already saved, skip
        # Check if it's a grid object (not yet saved)
        if isinstance(grid_info, dict) and 'grid' in grid_info:
            grids_to_save.append((source_name, grid_info))
    
    if not grids_to_save:
        log_message("All grids have already been saved.", 'info')
        error_display.value = "<span style='color: green;'>✅ All grids have already been saved.</span>"
        update_status("All grids saved", 100)
        return
    
    log_message(f"Saving {len(grids_to_save)} grid(s) to MongoDB...", 'info')
    update_status(f"Saving {len(grids_to_save)} grid(s)...", 0)
    
    if not voxel_storage or not mongo_client or not mongo_client.is_connected():
        log_message("MongoDB storage not available. Cannot save grids.", 'error')
        error_display.value = "<span style='color: red;'>❌ MongoDB storage not available. Cannot save grids.</span>"
        update_status("Storage unavailable", 0)
        return
    
    # Get model_id from dropdown (all grids should use the same model)
    model_id = None
    if model_dropdown.value and model_dropdown.value != "ALL":
        model_id = model_dropdown.value
    else:
        log_message("Please select a model first", 'warning')
        error_display.value = "<span style='color: red;'>❌ Please select a model first</span>"
        update_status("No model selected", 0)
        return
    
    # Get model name
    model_name = "Unknown"
    if stl_client and model_id:
        try:
            model_data = stl_client.get_model(model_id)
            if model_data:
                model_name = model_data.get('model_name') or model_data.get('filename') or model_data.get('original_stem', 'Unknown')
        except Exception as e:
            log_message(f"Error getting model name: {e}", 'warning')
    
    # Save all grids
    saved_count = 0
    failed_count = 0
    saved_grid_ids = {}
    
    try:
        for idx, (source_name, grid_info) in enumerate(grids_to_save):
            grid = grid_info['grid']
            grid_name = grid_info['grid_name']
            
            log_message(f"Saving {source_name} grid ({idx+1}/{len(grids_to_save)})...", 'info')
            update_status(f"Saving {source_name} grid...", 10 + (idx * 80 // len(grids_to_save)))
            
            # Validate that grid has required metadata
            if not hasattr(grid, 'metadata'):
                log_message(f"Error: {source_name} grid missing metadata. Skipping.", 'error')
                failed_count += 1
                continue
            
            # Get source from grid metadata (required - all grids must have this)
            source = grid.metadata.get('source', source_name)
            
            # Ensure all required metadata is present
            grid.metadata['model_id'] = model_id
            grid.metadata['source'] = source
            grid.metadata['grid_name'] = grid_name

            # Collect all configuration metadata - COMPREHENSIVE
            grid_type_val = grid_type.value if hasattr(grid_type, 'value') else (grid.grid_type if hasattr(grid, 'grid_type') else 'uniform')

            # Get resolution from grid (required for comprehensive metadata)
            actual_resolution = grid.resolution if hasattr(grid, 'resolution') else uniform_resolution.value

            config_metadata = {
                # CRITICAL: Source, grid_type, resolution (required for all operations)
                'source': source,
                'grid_type': grid_type_val,
                'resolution': actual_resolution,  # REQUIRED: Save resolution directly
                'stage': GridStage.EMPTY.value,
    
                # Grid type and mode
                'resolution_mode': resolution_mode.value if hasattr(resolution_mode, 'value') else 'uniform',
    
                # Resolution settings (detailed)
                'uniform_resolution': uniform_resolution.value if hasattr(uniform_resolution, 'value') and (resolution_mode.value == 'uniform' if hasattr(resolution_mode, 'value') else True) else actual_resolution,
                'x_resolution': x_resolution.value if hasattr(x_resolution, 'value') and (resolution_mode.value == 'per_axis' if hasattr(resolution_mode, 'value') else False) else None,
                'y_resolution': y_resolution.value if hasattr(y_resolution, 'value') and (resolution_mode.value == 'per_axis' if hasattr(resolution_mode, 'value') else False) else None,
                'z_resolution': z_resolution.value if hasattr(z_resolution, 'value') and (resolution_mode.value == 'per_axis' if hasattr(resolution_mode, 'value') else False) else None,
    
                # Bounding box mode
                'bbox_mode': bbox_mode.value if hasattr(bbox_mode, 'value') else 'model',
                'bbox_source': 'model' if (bbox_mode.value == 'model' if hasattr(bbox_mode, 'value') else True) else ('union' if (bbox_mode.value == 'union' if hasattr(bbox_mode, 'value') else False) else 'custom'),
    
                # Grid properties
                'aggregation_method': aggregation_method.value if hasattr(aggregation_method, 'value') else (grid.aggregation if hasattr(grid, 'aggregation') else 'mean'),
                'sparse_storage': sparse_storage.value if hasattr(sparse_storage, 'value') else False,
                'compression': compression.value if hasattr(compression, 'value') else False,
    
                # Adaptive settings (if applicable)
                'adaptive_strategy': adaptive_strategy.value if hasattr(adaptive_strategy, 'value') and (resolution_mode.value == 'adaptive' if hasattr(resolution_mode, 'value') else False) else None
            }
            
            # Add coordinate system if available
            if hasattr(coord_system, 'value'):
                config_metadata['coordinate_system'] = {
                    'type': coord_system.value,
                    'origin': [origin_x.value, origin_y.value, origin_z.value] if hasattr(origin_x, 'value') else [0, 0, 0],
                    'rotation': [rotation_x.value, rotation_y.value, rotation_z.value] if hasattr(rotation_x, 'value') else [0, 0, 0]
                }
            
            # Create description
            source_name_upper = source.upper()
            description = f"Empty {grid_type_val} grid for {source_name_upper} data - {model_name}"
            
            # Create tags
            tags = [
                source,
                grid_type_val,
                'empty',
                'notebook',
                'interactive',
                model_name if model_name != "Unknown" else 'unknown'
            ]
            
            # Save grid
            grid_id = voxel_storage.save_voxel_grid(
                model_id=model_id,
                grid_name=grid_name,
                voxel_grid=grid,
                description=description,
                tags=tags,
                model_name=model_name,
                configuration_metadata=config_metadata
            )
            
            # Update saved_grids with the saved grid info
            saved_grids[source_name] = {
                'grid_id': grid_id,
                'grid_name': grid_name,
                'model_id': model_id,
                'model_name': model_name,
                'grid_type': grid_type_val,
                'source': source,
                'created_at': datetime.now()
            }
            
            saved_grid_ids[source_name] = grid_id
            saved_count += 1
            log_message(f"✅ {source_name} grid saved successfully (ID: {grid_id[:8]}...)", 'success')
        
        # Update current_grid to the last saved grid
        if saved_count > 0:
            last_source = list(saved_grid_ids.keys())[-1]
            current_grid_id = saved_grid_ids[last_source]
            current_grid_model_id = model_id
            current_grid_model_name = model_name
            current_grid_name = saved_grids[last_source]['grid_name']
            # Set current_grid to the last saved grid's grid object
            if last_source in saved_grids and 'grid' in saved_grids[last_source]:
                current_grid = saved_grids[last_source]['grid']
        
        update_status("Save complete", 100)
        
        if saved_count == len(grids_to_save):
            log_message(f"✅ All {saved_count} grid(s) saved successfully!", 'success')
            error_display.value = f"<span style='color: green;'>✅ Successfully saved {saved_count} grid(s) to MongoDB</span>"
        else:
            log_message(f"⚠️ Saved {saved_count} grid(s), {failed_count} failed", 'warning')
            error_display.value = f"<span style='color: orange;'>⚠️ Saved {saved_count} grid(s), {failed_count} failed</span>"
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Save operation completed in {total_time:.2f}s", 'success')
        
        # Refresh grid dropdown to show the newly saved grids
        update_grid_dropdown()
        
    except Exception as e:
        log_message(f"Error saving grids: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error saving grids: {str(e)}</span>"
        update_status("Error saving grids", 0)
        
def update_grid_dropdown():
    """Update grid dropdown with empty grids for selected model, filtered by selected data sources."""
    global grid_dropdown
    
    if not voxel_storage or not mongo_client or not mongo_client.is_connected():
        grid_dropdown.options = [("━━━ MongoDB not available ━━━", None)]
        grid_dropdown.value = None
        return
    
    # Get selected model
    selected_model_id = model_dropdown.value
    if not selected_model_id or selected_model_id == "ALL":
        grid_dropdown.options = [("━━━ Select a specific model first ━━━", None)]
        grid_dropdown.value = None
        return
    
    # Get selected data sources for filtering
    selected_sources = get_selected_sources()
    all_sources = ['laser', 'ct', 'ispm', 'hatching']
    filter_by_source = len(selected_sources) < len(all_sources)  # Only filter if not all are selected
    
    try:
        # List all grids for this model
        available_grids = voxel_storage.list_grids(model_id=selected_model_id)
        
        # Filter to only empty grids (stage='empty') and by selected sources
        empty_grids = []
        for grid in available_grids:
            # Check metadata for stage
            metadata = grid.get('metadata', {})
            config_metadata = metadata.get('configuration_metadata', {})
            stage = config_metadata.get('stage', '')
            
            # Also check grid name for 'empty' stage
            grid_name = grid.get('grid_name', '')
            has_empty_stage = 'empty' in grid_name or stage == 'empty'
            
            # Check if grid has no signals (empty grid)
            available_signals = grid.get('available_signals', [])
            has_no_signals = len(available_signals) == 0
            
            # Must be an empty grid
            if not (has_empty_stage or has_no_signals):
                continue
            
            # Filter by source if not all sources are selected
            if filter_by_source:
                grid_source = config_metadata.get('source', 'unknown')
                # Also try to get source from grid name if not in metadata
                if grid_source == 'unknown':
                    grid_name_lower = grid_name.lower()
                    if 'laser' in grid_name_lower:
                        grid_source = 'laser'
                    elif 'ct' in grid_name_lower:
                        grid_source = 'ct'
                    elif 'ispm' in grid_name_lower:
                        grid_source = 'ispm'
                    elif 'hatching' in grid_name_lower:
                        grid_source = 'hatching'
                
                # Only include if source matches selected sources
                if grid_source not in selected_sources:
                    continue
            
            empty_grids.append(grid)
        
        # Build dropdown options
        grid_options = [("━━━ Select Empty Grid ━━━", None)]
        
        if not empty_grids:
            if filter_by_source:
                sources_str = ', '.join([s.upper() for s in selected_sources])
                grid_options.append((f"━━━ No empty grids found for {sources_str} ━━━", None))
                log_message(f"No empty grids found for selected sources: {', '.join(selected_sources)}", 'info')
            else:
                grid_options.append(("━━━ No empty grids found ━━━", None))
                log_message("No empty grids found for this model", 'info')
        else:
            # Group by source for better organization
            grids_by_source = {}
            for grid in empty_grids:
                metadata = grid.get('metadata', {})
                config_metadata = metadata.get('configuration_metadata', {})
                source = config_metadata.get('source', 'unknown')
                
                # Try to get source from grid name if not in metadata
                if source == 'unknown':
                    grid_name = grid.get('grid_name', '')
                    grid_name_lower = grid_name.lower()
                    if 'laser' in grid_name_lower:
                        source = 'laser'
                    elif 'ct' in grid_name_lower:
                        source = 'ct'
                    elif 'ispm' in grid_name_lower:
                        source = 'ispm'
                    elif 'hatching' in grid_name_lower:
                        source = 'hatching'
                
                if source not in grids_by_source:
                    grids_by_source[source] = []
                grids_by_source[source].append(grid)
            
            # Add grids grouped by source
            source_order = ['laser', 'ct', 'ispm', 'hatching']
            for source in source_order:
                if source in grids_by_source:
                    for grid in grids_by_source[source]:
                        # FIX: Use 'grid_id' instead of '_id'
                        grid_id = str(grid.get('grid_id', ''))
                        grid_name = grid.get('grid_name', 'Unknown')
                        
                        # Get additional info
                        metadata = grid.get('metadata', {})
                        config_metadata = metadata.get('configuration_metadata', {})
                        grid_type_val = config_metadata.get('grid_type', 'uniform')
                        resolution = config_metadata.get('resolution', 0.0)
                        
                        # Format label clearly
                        display_name = f"{source.upper()}: {grid_name}"
                        if resolution > 0:
                            display_name += f" ({grid_type_val}, {resolution}mm)"
                        display_name += f" [{grid_id[:8]}...]"
                        
                        grid_options.append((display_name, grid_id))
            
            # Add any grids with unknown source at the end
            if 'unknown' in grids_by_source:
                for grid in grids_by_source['unknown']:
                    # FIX: Use 'grid_id' instead of '_id'
                    grid_id = str(grid.get('grid_id', ''))
                    grid_name = grid.get('grid_name', 'Unknown')
                    
                    metadata = grid.get('metadata', {})
                    config_metadata = metadata.get('configuration_metadata', {})
                    grid_type_val = config_metadata.get('grid_type', 'uniform')
                    resolution = config_metadata.get('resolution', 0.0)
                    
                    display_name = f"UNKNOWN: {grid_name}"
                    if resolution > 0:
                        display_name += f" ({grid_type_val}, {resolution}mm)"
                    display_name += f" [{grid_id[:8]}...]"
                    
                    grid_options.append((display_name, grid_id))
        
        grid_dropdown.options = grid_options
        grid_dropdown.value = None  # Reset selection
        
        # Log success message
        if filter_by_source:
            sources_str = ', '.join([s.upper() for s in selected_sources])
            log_message(f"Found {len(empty_grids)} empty grid(s) for {sources_str} (filtered)", 'success')
        else:
            log_message(f"Found {len(empty_grids)} empty grid(s) for selected model", 'success')
        
    except Exception as e:
        grid_dropdown.options = [("━━━ Error loading grids ━━━", None)]
        log_message(f"Error loading grids: {str(e)}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error loading grids: {str(e)}</span>"
        
# ============================================
# Load Grid Function
# ============================================

def load_grid(button):
    """Load an empty grid from MongoDB for visualization."""
    global current_grid, current_grid_type, current_grid_id, current_grid_model_name, current_grid_model_id, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with grid_logs:
        clear_output(wait=True)
    
    log_message("Starting grid load operation...", 'info')
    update_status("Initializing load...", 0)
    
    if not voxel_storage or not mongo_client or not mongo_client.is_connected():
        log_message("MongoDB storage not available. Cannot load grid.", 'error')
        error_display.value = "<span style='color: red;'>❌ MongoDB storage not available. Cannot load grid.</span>"
        update_status("Storage unavailable", 0)
        return
    
    # Check if model is selected
    selected_model_id = model_dropdown.value
    if not selected_model_id or selected_model_id == "ALL":
        log_message("Please select a specific model first", 'warning')
        error_display.value = "<span style='color: orange;'>⚠️ Please select a specific model first (not 'All Models')</span>"
        update_status("Model selection required", 0)
        return
    
    # Get selected grid ID from dropdown
    selected_grid_id = grid_dropdown.value
    
    # Handle tuple format if needed
    if isinstance(selected_grid_id, tuple):
        selected_grid_id = selected_grid_id[1]
    
    # Validate selection
    if not selected_grid_id or selected_grid_id is None or selected_grid_id == '':
        log_message("Please select a grid from the dropdown menu", 'warning')
        error_display.value = "<span style='color: orange;'>⚠️ Please select an empty grid from the dropdown menu above</span>"
        update_status("Grid selection required", 0)
        return
    
    try:
        grid_id = str(selected_grid_id).strip()
        log_message(f"Loading grid ID: {grid_id[:8]}...", 'info')
        update_status(f"Loading grid {grid_id[:8]}...", 30)
        
        # Load the grid data
        loaded_grid = voxel_storage.load_voxel_grid(grid_id)
        
        if not loaded_grid:
            log_message(f"Failed to load grid {grid_id[:8]}...", 'error')
            error_display.value = f"<span style='color: red;'>❌ Failed to load grid {grid_id[:8]}...</span>"
            update_status("Error loading grid", 0)
            return
        
        # Verify it's an empty grid
        available_signals = loaded_grid.get('available_signals', [])
        metadata = loaded_grid.get('metadata', {})
        config_metadata = metadata.get('configuration_metadata', {})
        stage = config_metadata.get('stage', '')
        grid_name = loaded_grid.get('grid_name', '')
        source = config_metadata.get('source', 'unknown')
        
        # Get resolution from multiple possible locations
        resolution = None
        if 'resolution' in metadata:
            resolution = metadata.get('resolution')
        elif 'uniform_resolution' in config_metadata:
            resolution = config_metadata.get('uniform_resolution')
        elif 'resolution' in config_metadata:
            resolution = config_metadata.get('resolution')
        else:
            resolution = 1.0
            log_message("Resolution not found, using default: 1.0 mm", 'warning')
        
        log_message(f"Loaded grid: {grid_name} | Resolution: {resolution} mm | Source: {source}", 'info')
        
        is_empty = (len(available_signals) == 0) or (stage == 'empty') or ('empty' in grid_name)
        
        if not is_empty:
            log_message(f"⚠️ Warning: Grid {grid_id[:8]}... has signals. This notebook is for empty grids.", 'warning')
            error_display.value = f"<span style='color: orange;'>⚠️ This grid has {len(available_signals)} signal(s). Use Notebook 03 (Signal Mapping) to load mapped grids.</span>"
        
        log_message(f"Empty grid loaded: {grid_name} (source: {source}, resolution: {resolution}mm)", 'success')
        update_status("Processing grid data...", 50)
        
        # Extract metadata
        bbox_min = tuple(metadata.get('bbox_min', [0, 0, 0]))
        bbox_max = tuple(metadata.get('bbox_max', [1, 1, 1]))
        
        # Update global state
        current_grid_id = grid_id
        current_grid_model_name = loaded_grid.get('model_name', 'Unknown')
        current_grid_model_id = loaded_grid.get('model_id', selected_model_id)
        
        # Get grid type from metadata
        grid_type_from_meta = config_metadata.get('grid_type', 'uniform')
        current_grid_type = grid_type_from_meta if grid_type_from_meta in ['uniform', 'adaptive', 'multi'] else 'uniform'
        
        # Reconstruct VoxelGrid object for visualization
        current_grid = None
        
        try:
            if VOXEL_AVAILABLE:
                from am_qadf.voxelization.voxel_grid import VoxelGrid
                
                current_grid = VoxelGrid(
                    bbox_min=bbox_min,
                    bbox_max=bbox_max,
                    resolution=resolution,
                    aggregation='mean'
                )
                
                log_message(f"Grid structure reconstructed: {current_grid.dims} voxels, {resolution}mm resolution", 'success')
            else:
                class EmptyGridDisplay:
                    def __init__(self, bbox_min, bbox_max, resolution):
                        self.bbox_min = np.array(bbox_min)
                        self.bbox_max = np.array(bbox_max)
                        self.resolution = resolution
                        self.size = self.bbox_max - self.bbox_min
                        self.dims = np.ceil(self.size / resolution).astype(int)
                        self.dims = np.maximum(self.dims, [1, 1, 1])
                        self.actual_size = self.dims * resolution
                        self.available_signals = set()
                        self.voxels = {}
                
                current_grid = EmptyGridDisplay(bbox_min, bbox_max, resolution)
                log_message(f"Using display grid: {current_grid.dims} voxels, {resolution}mm resolution", 'info')
        
        except Exception as e:
            log_message(f"Error reconstructing grid: {e}", 'error')
            import traceback
            log_message(f"Traceback: {traceback.format_exc()}", 'error')
            error_display.value = f"<span style='color: red;'>❌ Error reconstructing grid: {e}</span>"
            update_status("Error", 0)
            return
        
        log_message("Updating displays...", 'info')
        update_status("Updating displays...", 80)
        
        # Update grid info display
        grid_info = f"""
        <p><b>✅ Empty Grid Loaded:</b></p>
        <ul>
            <li><b>Grid Name:</b> {grid_name}</li>
            <li><b>Grid ID:</b> {grid_id[:8]}...</li>
            <li><b>Source:</b> {source.upper()}</li>
            <li><b>Grid Type:</b> {current_grid_type}</li>
            <li><b>Resolution:</b> {resolution} mm</li>
            <li><b>Bounding Box:</b> {bbox_min} to {bbox_max}</li>
            <li><b>Dimensions:</b> {current_grid.dims if hasattr(current_grid, 'dims') else 'N/A'}</li>
            <li><b>Signals:</b> {len(available_signals)} (empty grid)</li>
        </ul>
        <p><b>💡 This is an empty grid structure ready for signal mapping in Notebook 03.</b></p>
        """
        status_display.value = grid_info
        
        # Update visualization
        update_status("Visualizing grid...", 90)
        update_visualization()
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Grid loaded and visualized in {total_time:.2f}s", 'success')
        
        update_status("Grid loaded successfully", 100)
        
    except Exception as e:
        log_message(f"Error loading grid: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error loading grid: {str(e)}</span>"
        update_status("Error loading grid", 0)
        
# ============================================
# Visualization Update Function
# ============================================
def update_visualization():
    """Update visualization based on current viz_mode."""
    if viz_mode.value == '3d':
        update_3d_view()
    elif viz_mode.value == '2d':
        update_slice_view()
    else:  # props
        update_properties_display()

# Connect events
create_button.on_click(create_grid)
load_button.on_click(load_grid)
save_button.on_click(save_grid)
slice_axis.observe(lambda x: update_slice_view(), names='value')
slice_position.observe(lambda x: update_slice_view(), names='value')
model_dropdown.observe(lambda x: update_grid_dropdown(), names='value')
refresh_grids_button.on_click(lambda b: update_grid_dropdown())

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)

## Summary

Congratulations! You've learned how to create and configure voxel grids.

### Key Takeaways

1. **Grid Types**: Uniform, Adaptive, and Multi-Resolution grids for different use cases
2. **Bounding Box**: Configure spatial extent from models or custom coordinates
3. **Resolution**: Set uniform, per-axis, or adaptive resolution
4. **Coordinate Systems**: Transform between machine, part, and build coordinate systems
5. **Visualization**: View grids in 3D, 2D slices, or property tables

### Next Steps

Proceed to:
- **03_Signal_Mapping_Fundamentals.ipynb** - Learn to map signals to voxel grids
- **04_Temporal_and_Spatial_Alignment.ipynb** - Learn synchronization and alignment

### Related Resources

- Voxelization Module Documentation: `../docs/AM_QADF/05-modules/voxelization.md`
- API Reference: `../docs/AM_QADF/06-api-reference/voxelization-api.md`
- Examples: `../examples/`
